In [17]:
import numpy as np
import pandas as pd
import spacy
import nltk
import json
import re
from collections import Counter

### Reading the training data and pre-processing the text(lower casing, lead/train space stripping,remove punctuations)

In [18]:
train_data_file = 'train-data-prepared.json'
df_train = pd.read_json(train_data_file)
# display(df_train)

In [21]:
df_train['clean_text'] = df_train['text'].str.lower()
df_train['clean_text'] = df_train['clean_text'].str.strip()
df_train['clean_text'] = df_train['clean_text'].str.replace('[^\w\s]','')
display(df_train)

,id,text,label,clean_text
0,1363ac6f16e232a1d8e9343d975ebe10,Since communism has been relegated to just a h...,0,since communism has been relegated to just a h...
1,6b925ce5eeb8b690b35972abafcb7c60,Can you counter that?,0,can you counter that
2,99977c6e63734add1c1b600be79b3342,Censorship does not eliminate the censored ind...,0,censorship does not eliminate the censored ind...
3,7e34d9e198bc9e12f0868793c68d32f0,"Without the extra population from abortions, h...",0,without the extra population from abortions ha...
4,629d09668c3339dba831f6c81a307b0e,I can't stand it,1,i cant stand it
...,...,...,...,...
2614,452684ca088e4ab1c58d89e1a28e1ef7,"it's ""true"" or not and that ""truth"" is availab...",0,its true or not and that truth is available to...
2615,b5fa34bde09f97ab565ec1ae433d1797,And these slogans don't even denote any sense ...,0,and these slogans dont even denote any sense o...
2616,f676671baa396678dcb3471ea67e70ed,&gt;whole-bodyWhile,0,gtwholebodywhile
2617,a4e0fa2814bb40ae76750ea1597084de,that the majority of them are affected negativ...,1,that the majority of them are affected negativ...


In [22]:
nlp = spacy.load("en_core_web_sm")

In [48]:
# Extracting token count and adding it as a DF column

df_train['token_count'] = df_train['clean_text'].apply(lambda x: len(nlp(x)))

In [25]:
display(df_train)

,id,text,label,clean_text,tokens
0,1363ac6f16e232a1d8e9343d975ebe10,Since communism has been relegated to just a h...,0,since communism has been relegated to just a h...,11
1,6b925ce5eeb8b690b35972abafcb7c60,Can you counter that?,0,can you counter that,4
2,99977c6e63734add1c1b600be79b3342,Censorship does not eliminate the censored ind...,0,censorship does not eliminate the censored ind...,7
3,7e34d9e198bc9e12f0868793c68d32f0,"Without the extra population from abortions, h...",0,without the extra population from abortions ha...,38
4,629d09668c3339dba831f6c81a307b0e,I can't stand it,1,i cant stand it,5
...,...,...,...,...,...
2614,452684ca088e4ab1c58d89e1a28e1ef7,"it's ""true"" or not and that ""truth"" is availab...",0,its true or not and that truth is available to...,15
2615,b5fa34bde09f97ab565ec1ae433d1797,And these slogans don't even denote any sense ...,0,and these slogans dont even denote any sense o...,22
2616,f676671baa396678dcb3471ea67e70ed,&gt;whole-bodyWhile,0,gtwholebodywhile,1
2617,a4e0fa2814bb40ae76750ea1597084de,that the majority of them are affected negativ...,1,that the majority of them are affected negativ...,12


In [54]:
text = "him personally feel would feel death penalty should be abolished"
doc1 = nlp(text)
md_count = 0
for token in doc1:
    print(token.pos_, token.tag_)
    if token.tag_ == 'MD':
        md_count = md_count + 1

print(md_count)

PRON PRP
ADV RB
VERB VBP
VERB MD
VERB VB
NOUN NN
NOUN NN
VERB MD
VERB VB
VERB VBN
2


In [ ]:
for text in df['clean_text']:
    doc = nlp(text)
    for token in doc:
        if token_tag = 'MD'
    
    
    